# Saving images in issues

We often create issues to collectively share images relating to a project.

Reincarnation has a nice example. https://github.com/deathbeds/reincarnation/issues/1

In [87]:
import pandas
from CommonMark.render.renderer import Renderer
from CommonMark import Parser
from ipywidgets import *

from dataclasses import dataclass

from IPython.utils.capture import capture_output

In [75]:
class ImageFinder(Renderer):
    def image(self, node, entering): self.out(node.destination + '\n')

`scrape_images_from_issue` will take {repo} and issue {id}.  It will return the images in the markdown.

In [88]:
@dataclass(order=True)
class IssueScraper:
    repo: str = None
    id: int = None
    df: pandas.DataFrame = None
    
    def __post_init__(self):
        if self.df is None: 
            self.df = pandas.read_json(
                f"""https://api.github.com/repos/{self.repo}/issues/{self.id}/comments""")
        
    def images(self): return self.df.body.apply(
            lambda x: ImageFinder().render(Parser().parse(x))
        ).apply(str.splitlines).apply(pandas.Series).stack().drop_duplicates()
    
    def _repr_markdown_(self): return ''.join(map('![]({0})\n'.format, self.images()))
    
    def _repr_mimebundle_(self, **_):
        selector, image = Select(options=list(self.images())), HTML()
        dlink((selector, 'value'), (image, 'value'), "<img src='{}' width='100%'/>".format)
        with capture_output() as out: HBox(children=[selector, image])._ipython_display_()
        return {
            **out.outputs[0].data,
            'text/markdown': self._repr_markdown_()}, {}

In [89]:
def _sample_issues_from_reincarnation():
    return IssueScraper('deathbeds/reincarnation', 1)

In [ ]:
def _get_reincarnation_images():
    return scrape_images_from_issue('deathbeds/reincarnation', 1)